In [ ]:
#ensembling
from mlxtend.classifier import StackingCVClassifier
import numpy as np
import pandas as pd
import sys, os, glob
import imp
import seaborn as sns
sys.path.append('./src/')
import common_utils,my_ensembler, feature_utils, defines, model_utils, my_bert
from sklearn.neighbors import KNeighborsClassifier
from sklearn_crfsuite import scorers, CRF
from sklearn.pipeline import Pipeline
import json
from operator import itemgetter
from sklearn_crfsuite.utils import flatten
from sklearn_crfsuite.metrics import sequence_accuracy_score
from sklearn.linear_model import LogisticRegression
import scipy
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
import classes
import pickle

In [ ]:
dir_name='reparse'
seq_len=15
step=15
cv_split_name='cv_splits_10'
merged_str='scaled.binary0.50_merged'
doc_indices=np.arange(1,81)
doc_indices
test_doc_num=8
seed=42

### Dataset without 76 document

In [ ]:
imp.reload(classes)
imp.reload(feature_utils)
dataset=classes.Dataset(dir_name=dir_name,doc_indices=doc_indices,merged_str=merged_str)
dataset.set_tf_params('lemma')
dataset.set_tf_params('word')
dataset.set_tf_params('char_wb')
dataset.pack_dataset()
dataset.pack_sent_per_paragraph()
dataset.reshape(seq_len,step)
dataset.dump_to_file("new.76")

### Check retrieving the paragraph for cross-validation

In [ ]:
imp.reload(model_utils)
paragraph_column=dataset.get_paragraph([1], '15_15')
paragraph_column

In [ ]:
len(flatten(paragraph_column))

In [ ]:
demo_dataset.reshape(seq_len,step)


In [ ]:
demo_dataset.dump_to_file("no.76")

### Load splits of documents as were used for BERT

In [11]:
cv_splits_loaded=common_utils.load_json(dir_name,cv_split_name)
cv_splits_loaded

Opened /scratch/sim_reg6/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/reparse/cv_splits_10.json


{'0': {'test': [4, 14, 15, 18, 29, 32, 36, 70],
  'train': [1,
   2,
   3,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   16,
   17,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   30,
   31,
   33,
   34,
   35,
   37,
   38,
   39,
   40,
   41,
   42,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   51,
   52,
   53,
   54,
   55,
   56,
   57,
   58,
   59,
   60,
   61,
   62,
   63,
   64,
   65,
   66,
   67,
   68,
   69,
   71,
   72,
   73,
   74,
   75,
   76,
   77,
   78,
   79,
   80]},
 '1': {'test': [5, 6, 13, 34, 37, 62, 73, 80],
  'train': [1,
   2,
   3,
   4,
   7,
   8,
   9,
   10,
   11,
   12,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   35,
   36,
   38,
   39,
   40,
   41,
   42,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   51,
   52,
   53,
   54,
   55,
   56,
   57,
   58,
   59,
   60,
   61,
   6

In [18]:
tmp_split={'0':{'test':[1],'train':[2]}, '1': {'test':[3],'train':[4]}}

### Run cross-validation

In [32]:
imp.reload(model_utils)
cv_db_crf=model_utils.prepared_cross_validate_crf(dataset, cv_splits_loaded, seq_len=seq_len, step=step)

crf_params not passed
0 split started for 72 train sequences...
0 split fit of 229 samples took 00:00:27
0 split predict took 00:00:00
1 split started for 72 train sequences...
1 split fit of 267 samples took 00:00:26
1 split predict took 00:00:00
2 split started for 72 train sequences...
2 split fit of 231 samples took 00:00:27
2 split predict took 00:00:00
3 split started for 72 train sequences...
3 split fit of 280 samples took 00:00:26
3 split predict took 00:00:00
4 split started for 72 train sequences...
4 split fit of 258 samples took 00:00:27
4 split predict took 00:00:00
5 split started for 72 train sequences...
5 split fit of 280 samples took 00:00:25
5 split predict took 00:00:00
6 split started for 72 train sequences...
6 split fit of 227 samples took 00:00:26
6 split predict took 00:00:00
7 split started for 72 train sequences...
7 split fit of 284 samples took 00:00:27
7 split predict took 00:00:00
8 split started for 72 train sequences...
8 split fit of 310 samples took 

In [33]:
cv_db_crf

,crf_group,crf_par,crf_split,crf_predicted,crf_true,crf_proba_0,crf_proba_1,crf_sent_idx
0,4,0,0,not_nar,not_nar,0.977135,0.022865,0
1,4,1,0,not_nar,not_nar,0.976613,0.023387,1
2,4,2,0,not_nar,not_nar,0.986431,0.013569,2
3,4,3,0,not_nar,not_nar,0.976959,0.023041,3
4,4,3,0,not_nar,not_nar,0.948269,0.051731,4
...,...,...,...,...,...,...,...,...
38776,79,171,9,not_nar,not_nar,0.888592,0.111408,3773
38777,79,172,9,not_nar,not_nar,0.996061,0.003939,3774
38778,79,172,9,not_nar,not_nar,0.999550,0.000450,3775
38779,79,172,9,not_nar,not_nar,0.999665,0.000335,3776


In [ ]:
crf_scores, crf_dict = model_utils.get_report_from_splits(cv_db_crf,"crf")

In [ ]:
crf_dict


In [ ]:
crf_scores,np.mean(crf_scores)

In [ ]:
worst_split=np.argmin(crf_scores)
worst_split

In [ ]:
f1={}
f1['is_nar']=[]
f1['not_nar']=[]
for label in f1.keys():
    for key,val in crf_dict.items():
        f1[label].append(val[label]['f1-score'])

In [ ]:
for k,v in f1.items():
    print(k,np.mean(v))

In [ ]:
colors=['r','b']
linestyle=['dashed','solid']

In [ ]:
from matplotlib import pyplot as plt
for y, c, st in zip(f1.keys(), colors,linestyle):
    data=f1[y]
    min_x = np.argmin(data)
    min_y = np.min(data)
    plt.plot(data,marker='o',color=c,linestyle=st,label=y)#, color='magenta', marker='o',mfc='pink' ) #plot the data
#     plt.scatter(min_x, min_y,c='r', label='minimum',zorder =1)
plt.xticks(range(0,len(data), 1)) #set the tick frequency on x-axis
plt.legend()
plt.ylabel('f-1 score') #set the label for y axis
plt.xlabel('fold index') #set the label for x-axis
plt.title('f-1 score per label') #set the title of the graph
plt.show() #display the graph

In [ ]:
f1

### Try single example

### Search parameters on prepared dataset with prepared splits (same as for BERT)

In [ ]:
dataset  = classes.Dataset(dir_name=dir_name,doc_indices=doc_indices,merged_str=merged_str)
dataset_loaded=pickle.load( open( os.path.join(dataset.path,"dataset_full.p"), "rb" ) )

In [ ]:
imp.reload(model_utils)
imp.reload(classes)

X,y,g,_ =  model_utils.get_X_y_by_doc_indices(dataset, doc_indices, seq_len, step)

In [ ]:
X[0]

In [ ]:
doc_indices

In [ ]:
cv_splits_loaded['0'].keys()

In [ ]:
type(cv_splits_loaded)

In [ ]:
imp.reload(classes)
cv_split_classes = []
for key,val in cv_splits_loaded.items():
    cv_split_classes.append(classes.Split(val['train'],val['test']))
cv_split_classes

In [ ]:
imp.reload(model_utils)
gsf = model_utils.GroupSplitFold(n_splits=len(cv_split_classes),prepared_splits=cv_split_classes)

for tr,ts in gsf.split():
    print(ts)

In [ ]:
imp.reload(model_utils)
my_crf = model_utils.CrfClassifier(crf_model=CRF(
                max_iterations=100,
                all_possible_transitions=True))

In [ ]:
my_crf.find_best_params(X=X,y=y,groups=g,cv=gsf,random_state=seed)

### First interation of best params

In [ ]:
params={}

In [ ]:
params[0]= {'algorithm': ['lbfgs'],
        'c1': scipy.stats.expon(scale=0.5),
        'c2': scipy.stats.expon(scale=0.05),
        'min_freq': np.arange(1, 8, 2)}


In [ ]:
my_crf.find_best_params(X=X,y=y,groups=g,cv=gsf,random_state=seed,**params[0])

In [ ]:
vars(my_crf)

### Use f1_score + WinPR for parameters search

In [ ]:
imp.reload(classes)
mixed_scorer = classes.MyScorer(window_size=40)

In [ ]:
def score_func(y_true,y_pred):
    win_score = win_f1.score_func(y_true,y_pred)
    label_score = metrics.flat_f1_score(y_true,y_pred)
    print("winPR {} label {}".format(win_score,label_score))
    mixed_score = np.average(a=[win_score,label_score],weights=[0.5,0.5])
    return mixed_score

In [ ]:
y=[[1,1],[0,0]]
y_p=[[1,0],[0,0]]
score_func(y,y_p)

In [ ]:
crf_mixed_score=model_utils.CrfClassifier(
                crf_model=CRF(
                max_iterations=100,
                all_possible_transitions=True),
#                 scorer=score_func)
    scorer= mixed_scorer.score_func)

In [ ]:
vars(crf_mixed_score)

In [ ]:
X_demo,y_demo,g_demo,_ =  model_utils.get_X_y_by_doc_indices(demo_reloaded_dataset, [1,2], seq_len, step)

In [ ]:
imp.reload(model_utils)
imp.reload(feature_utils)
imp.reload(classes)
gsf_demo=model_utils.GroupSplitFold(n_splits=2, n_groups=1)
for tr,ts in gsf_demo.split(X=[1,2],groups=[1,2]):
    print(ts,tr)

In [ ]:
crf_mixed_score.find_best_params(X=X_demo,y=y_demo,groups=g_demo,cv=gsf_demo,random_state=seed,**params[0])

<b>put aside mixed measure</b>

In [ ]:
params[1]= {'algorithm': ['lbfgs'],
        'c1': np.arange(0.45,0.49,0.01),
        'c2': np.arange(0.008,0.01,0.001),
        'min_freq': np.arange(4, 7, 1)}

In [ ]:
my_crf.find_best_params(X=X,y=y,groups=g,cv=gsf,random_state=seed,**params[1])

In [ ]:
params[4]= {'algorithm': ['lbfgs'],
        'c1': np.arange(0.459,0.461,0.001),
        'c2': np.arange(0.0069,0.0072,0.0001),
        'min_freq': [5]}

In [ ]:
my_crf.find_best_params(X=X,y=y,groups=g,cv=gsf,random_state=seed,**params[4])

In [ ]:
params[5]= {'algorithm': ['lbfgs'],
        'c1': scipy.stats.expon(scale=0.459),
        'c2': scipy.stats.expon(scale=0.0071),
        'min_freq': [5]}

In [ ]:
my_crf.rs[4].best_score_

In [ ]:
my_crf.find_best_params(X=X,y=y,groups=g,cv=gsf,random_state=seed,**params[5])

In [ ]:
common_utils.dump_to_file(my_crf.rs[4].best_params_,"crf.rs.params.793")

### Cross-validate CRF on found best parameters

In [ ]:
imp.reload(model_utils)
cv_db_best_params = pd.DataFrame()
cv_db=pd.DataFrame()
cv_db_best_params=model_utils.prepared_cross_validate_crf(cv_db,dataset, cv_splits_loaded, seq_len, step,**my_crf.rs[4].best_params_)

In [ ]:
crf_scores_best, crf_dict_best = model_utils.get_report_from_splits(cv_db_best_params,"crf")

In [ ]:
crf_scores_best,np.mean(crf_scores_best)

In [ ]:
common_utils.save_db(cv_db_best_params,dir_name,"cv_crf_best_params.78")

### Try WinPR score

In [ ]:
imp.reload(classes)
wd=classes.WinPR(40)

def score_func(x):
    return wd.score_func(x['crf_true'].tolist(),x['crf_predicted'].tolist())

cv_db_crf.groupby('crf_group').apply(score_func)

In [ ]:
imp.reload(model_utils)
imp.reload(classes)
my_crf = model_utils.CrfClassifier(crf_model=CRF(
                max_iterations=100,
                all_possible_transitions=True),
                scorer=wd.score_func)

In [ ]:
params_winpr= {'algorithm': ['lbfgs'],
        'c1': scipy.stats.expon(scale=0.5),
        'c2': scipy.stats.expon(scale=0.05),
        'min_freq': np.arange(1, 8, 2)}

In [ ]:
my_crf.find_best_params(X=X,y=y,groups=g,cv=gsf,random_state=seed,**params_winpr)

## Calculate F1 for paragraphs

In [ ]:
imp.reload(classes)
dataset_par  = classes.Dataset(dir_name=dir_name,doc_indices=doc_indices,merged_str=merged_str)

### Copy docs map from old dataset to a new one that has implemented get_paragpraph()

In [ ]:
dataset_par.doc_map = dataset.doc_map.copy()

In [ ]:
vars(dataset_par)

### Add paragraph index to result database

In [ ]:
def add_paragraph(x):
    doc_idx=x['crf_group'].unique()
    dataset_par.get_paragraph([doc_idx],'')

In [ ]:
cv_db_crf.groupby('crf_group').apply(add_paragraph)

In [ ]:
dataset

In [ ]:
crf_try = CRF(
            max_iterations=100,
            all_possible_transitions=True,
            algorithm='lbfgs'
        ).fit(X_train, y_train)

In [ ]:
y_pred=crf_try.predict(X_test)

In [ ]:
feature_utils.get_prediction_report(flatten(y_test), flatten(
                y_pred), crf.classes_)

### Define static parameters for data

In [ ]:
dir_name='reparse'
docs_map_name = 'scaled.binary.5_stop_docs_map.json'
crf_params_name='.802_21.07_07:26_best_params.json'
ens_crf_db_name   = 'ensemble_crf_cv_db_all'
ens_lg_db_name = 'ensemble_lg_cv_db_all'
cv_crf_bert_name = 'cv_crf_bert_all'
cv_split_name='cv_splits_10'
per_par=True
seq_len=3
step=3
test_doc_num=8
n_splits=10
seed=42

### Load docs map

In [ ]:
imp.reload(common_utils)
docs_map=common_utils.get_docs_map(dir_name,docs_map_name,per_par,seq_len,step)

In [ ]:
imp.reload(model_utils)
my_crf=model_utils.CrfClassifier(crf)
# my_crf.find_best_params(X=x_tr,y=y_tr,groups=g_tr)

In [ ]:
my_crf.predict_on_best_params(0,X=x_ts,y=y_ts)

### Use same splits as for CRF & BERT (10 x 10)

In [ ]:
x=np.arange(1,81)
cv_splits_10=model_utils.get_test_train_splits(x,test_doc_num,n_splits,seed)
cv_splits_10

In [ ]:
imp.reload(model_utils)
cv_splits=model_utils.get_test_train_splits(doc_indices=doc_indices,test_doc_num=2,n_splits=2,seed=42)
cv_splits

In [ ]:
n_splits=10
gsf = model_utils.GroupSplitFold(n_splits=n_splits, n_groups=test_doc_num)

for tr,ts in gsf.split(X=x, groups=x, seed=seed):
    print(ts)

In [ ]:
imp.reload(classes)
dataset.get_x([1,3],'15_15')

In [ ]:
X,y,groups=model_utils.get_X_y_by_doc_indices(docs_map,list(docs_map.keys()),seq_len,step)

In [ ]:
tot_len=0
check=X
for l in check:
    if isinstance(l,list):
        tot_len+=len(check)
tot_len

In [ ]:
rs = {}

In [ ]:
vars(dataset.doc_map[1])

In [ ]:
global iterr

In [ ]:
def increment_iter():
    global iterr
    iterr+=1
    print('шtter {}'.format(iterr))

In [ ]:
%%time
# define fixed parameters and parameters to search
increment_iter()
crf = CRF(
    max_iterations=100,
    all_possible_transitions=True,
    algorithm='lbfgs'
)
c1_param_range = np.arange(0.798,0.802,0.001)
c2_param_range = np.arange(0.0078,0.0082,0.0001)

params_space = {
    'c1': c1_param_range,
    'c2': c2_param_range,
    'min_freq': np.arange(1,4,1),
#      'epsilon':scipy.stats.expon(scale=0.00005),
#      'delta':scipy.stats.expon(scale=0.00005)
}

labels = ['not_nar','is_nar']
# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=labels)

# search

rs[iterr] = GridSearchCV(crf, params_space,
                        cv=gsf,
                        verbose=3,
                        n_jobs=-1,
                        scoring=f1_scorer)

rs[iterr].fit(X=X, y=y,groups=groups)

In [ ]:
# grid search
print('best params:', rs[iterr].best_params_)
print('best CV score:', rs[iterr].best_score_)
print('model size: {:0.2f}M'.format(rs[iterr].best_estimator_.size_ / 1000000))

In [ ]:
rs[0].best_params_,rs[0].best_score_

In [ ]:
imp.reload(common_utils)
common_utils.save_best_params(rs[iterr].best_params_, rs[iterr].best_score_,dir_name)

In [ ]:
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

In [ ]:
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

In [ ]:
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

In [ ]:
import json
from datetime import datetime

# datetime object containing current date and time
now = datetime.now()
dt_string = now.strftime("%d.%m_%H:%M")
print("date and time =", dt_string)
score="{:.3f}".format(rs.best_score_).lstrip('0')
score
json_path = os.path.join(os.getcwd(),defines.PATH_TO_DFS,dir_name,"{}_{}_best_params.json".format(score,dt_string))

best_params_pyth = common_utils.convert_to_python_types(rs.best_params_)
json_path
with open(json_path, 'w') as fp:
    json.dump(best_params_pyth, fp)

In [ ]:
json_path